In [1]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [2]:
def get_similarity(w1, w2, embeddings, size):
    try:
        w1_vector = embeddings[w1]
        w2_vector = embeddings[w2]
    except KeyError:
        return -10
    
    w1_neighbours_vectors = embeddings.similar_by_vector(w1_vector, topn = size)
    w2_neighbours_vectors = embeddings.similar_by_vector(w2_vector, topn = size)
    
    w1_neighbour_words = [embed[0] for embed in w1_neighbours_vectors]
    w2_neighbour_words = [embed[0] for embed in w2_neighbours_vectors]
    
    w1_cosine = np.average([cosine_similarity([w1_vector], [embeddings[w2]]) for w2 in w2_neighbour_words])
    w2_cosine = np.average([cosine_similarity([w2_vector], [embeddings[w1]]) for w1 in w1_neighbour_words])
    
    return np.average([w1_cosine, w2_cosine])

In [3]:
source = '../data/experiment_final.csv'
pd_dataset = pd.read_csv(source)
pd_dataset = pd_dataset.sort_values(by = 'language', ascending = True)
pd_dataset.head()

,c,i,id,l1_c,l1_i,language,type,code
2827,plans,projects,person_2799,plans,projectes,Catalan,RN,ca
2733,waste,lose,person_2421,malgastar,perdre,Catalan,RV,ca
2732,ending,end,person_1465,final,final,Catalan,RN,ca
2731,shot,view,person_1465,disparat,mostra,Catalan,RN,ca
2730,do,join,person_1465,fer,unir,Catalan,RV,ca


In [41]:
def get_embedding_file_name(language, file_type):
    file_name = file_type + '.'
    if (language == "Russian"):
        file_name += 'ru'
    elif (language == "Turkish"):
        file_name += 'tr'
    elif (language == "German"):
        file_name += 'de'
    elif (language == "Japanese"):
        file_name += 'ja'
    elif (language == "Spanish"):
        file_name += 'es'
    elif (language == "Polish"):
        file_name += 'pl'
    elif (language == "Italian"):
        file_name += 'it'
    elif (language == "Catalan"):
        file_name += 'ca'
    elif (language == "Korean"):
        file_name += 'ko'
    elif (language == "French"):
        file_name += 'fr'
    elif (language == "Chinese" or language == "Chinese (Simplified)"):
        file_name += 'zh'
    elif (language == "Portuguese"):
        file_name += 'pt'
    elif (language == "Swedish"):
        file_name += 'sv'
    elif (language == "Greek"):
        file_name += 'el'
    elif (language == "Thai"):
        file_name += 'th'
    elif (language == "Dutch"):
        file_name += 'nl'
    elif (language == "English"):
        file_name += 'en.300'
        
    file_name += '.vec' 
    return file_name

def load_embedding_vector(file_name):
    return KeyedVectors.load_word2vec_format(embedding_path + file_name, binary=False, unicode_errors='ignore')  # C text format

In [5]:
neighbour_list = [5, 10, 25, 50, 75, 100]
embedding_list = ['cc']
embedding_path = '../data/pretrained_embeddings/'

In [6]:
def run_experiment_english(dataframe):
    for vector_type in embedding_list:
        vec_type_start = datetime.now()
        embedding_file_name = get_embedding_file_name("English", vector_type)
        print('Loading embeddings: ' + embedding_file_name)
        embeddings = load_embedding_vector(embedding_file_name)
        print('Embeddings ' + embedding_file_name + ' loaded in ' + str(datetime.now() - vec_type_start))
        for neighbour_size in neighbour_list:
            neighbour_size_start = datetime.now()
            l2_sim = []
            for ind, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):
                w1 = row['c']
                w2 = row['i']
                l2_sim.append(get_similarity(w1, w2, embeddings, neighbour_size))
            dataframe['l2_sim_' + vector_type + '_' + str(neighbour_size)] = l2_sim
            print('Finished run for neighbour size: ' + str(neighbour_size) + ' in ' + str(datetime.now() - neighbour_size_start))
        print('Finished run for vector type: ' + vector_type + ' in ' + str(datetime.now() - vec_type_start))

In [7]:
run_experiment_english(pd_dataset)

Loading embeddings: cc.en.300.vec


  0%|          | 0/4937 [00:00<?, ?it/s]

Embeddings cc.en.300.vec loaded in 0:07:06.017325


  0%|          | 1/4937 [00:00<13:25,  6.13it/s]

Finished run for neighbour size: 5 in 0:11:50.284184


...

Finished run for neighbour size: 50 in 0:13:32.924132


  0%|          | 0/4937 [00:00<?, ?it/s]

Finished run for neighbour size: 75 in 0:14:34.614367


100%|██████████| 4937/4937 [15:35<00:00,  5.28it/s]


Finished run for neighbour size: 100 in 0:15:35.121350
Finished run for vector type: cc in 1:27:12.255671


In [10]:
pd_dataset.to_csv('../data/ms_final_exnglish_experiments2.csv', index = False)

In [20]:
language_list = pd_dataset['language'].unique()
# pd_dataset[pd_dataset['language'] == 'Catalan'].count()
language_list

array(['Catalan', 'Chinese (Simplified)', 'Dutch', 'French', 'German',
       'Greek', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese',
       'Russian', 'Spanish', 'Swedish', 'Thai', 'Turkish'], dtype=object)

In [26]:
def get_experiment_model(language, vector_type):
    vec_type_start = datetime.now()
    embedding_file_name = get_embedding_file_name(language, vector_type)
    print('Loading embeddings: ' + embedding_file_name)
    embeddings = load_embedding_vector(embedding_file_name)
    print('Embeddings ' + embedding_file_name + ' loaded in ' + str(datetime.now() - vec_type_start))
    return embeddings
new_columns = {'5': [], '10': [], '25': [], '50': [], '75': [], '100': []}
def run_experiment_l1(dataframe):
    current_lang = ''
    current_vector_type = ''
    
    for vector_type in embedding_list:
        vec_type_start = datetime.now()
        for language in language_list:
            language_start = datetime.now()
            embeddings = get_experiment_model(language, vector_type)
            for neighbour_size in neighbour_list:
                neighbour_size_start = datetime.now()
#                 l1_sim = []
                dset = dataframe[dataframe['language'] == language]
                for ind, row in tqdm(dset.iterrows(), total=dset.shape[0]):
                    w1 = row['l1_c']
                    w2 = row['l1_i']
                    new_columns[str(neighbour_size)].append(get_similarity(w1, w2, embeddings, neighbour_size))
#                 dataframe['l1_sim_' + vector_type + '_' + str(neighbour_size)] = l1_sim 
                print('Finished run for neighbour size: ' + str(neighbour_size) + ' in ' + str(datetime.now() - neighbour_size_start))
            print('Finished run for language: ' + language + ' in ' + str(datetime.now() - language_start))
        print('Finished run for vector type: ' + vector_type + ' in ' + str(datetime.now() - vec_type_start))

In [27]:
run_experiment_l1(pd_dataset)

Loading embeddings: cc.ca.vec


  0%|          | 0/325 [00:00<?, ?it/s]

Embeddings cc.ca.vec loaded in 0:07:09.292250


  0%|          | 1/325 [00:00<00:49,  6.54it/s]

Finished run for neighbour size: 5 in 0:00:55.072082


...

  0%|          | 0/192 [00:00<?, ?it/s]

Finished run for neighbour size: 75 in 0:00:33.706907


100%|██████████| 192/192 [00:36<00:00,  5.32it/s]


Finished run for neighbour size: 100 in 0:00:36.078619
Finished run for language: Japanese in 0:10:18.938850
Loading embeddings: cc.ko.vec


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte

In [30]:
for key in new_columns:
    print(str(key) + ':' + str(len(new_columns[key])))

5:2599
10:2599
25:2599
50:2599
75:2599
100:2599


In [31]:
pending_langs = ['Korean', 'Polish', 'Portuguese', 'Russian', 'Spanish', 'Swedish', 'Thai', 'Turkish']

In [32]:
test = KeyedVectors.load_word2vec_format(embedding_path + 'cc.ko.vec', binary=False)  # C text format

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte

In [39]:
back_up_col = {}
for key in new_columns.keys():
    if (key not in back_up_col):
        back_up_col[key] = []
    for row in new_columns[key]:
        back_up_col[key].append(str(row))

In [40]:
import json

with open('back_up_tex.txt', 'w') as file:
     file.write(json.dumps(back_up_col)) # 

In [43]:
def run_experiment_l1(dataframe):
    current_lang = ''
    current_vector_type = ''
    
    for vector_type in embedding_list:
        vec_type_start = datetime.now()
        for language in pending_langs:
            language_start = datetime.now()
            embeddings = get_experiment_model(language, vector_type)
            for neighbour_size in neighbour_list:
                neighbour_size_start = datetime.now()
#                 l1_sim = []
                dset = dataframe[dataframe['language'] == language]
                for ind, row in tqdm(dset.iterrows(), total=dset.shape[0]):
                    w1 = row['l1_c']
                    w2 = row['l1_i']
                    new_columns[str(neighbour_size)].append(get_similarity(w1, w2, embeddings, neighbour_size))
#                 dataframe['l1_sim_' + vector_type + '_' + str(neighbour_size)] = l1_sim 
                print('Finished run for neighbour size: ' + str(neighbour_size) + ' in ' + str(datetime.now() - neighbour_size_start))
            print('Finished run for language: ' + language + ' in ' + str(datetime.now() - language_start))
        print('Finished run for vector type: ' + vector_type + ' in ' + str(datetime.now() - vec_type_start))

In [44]:
run_experiment_l1(pd_dataset)

Loading embeddings: cc.ko.vec


  0%|          | 0/185 [00:00<?, ?it/s]

Embeddings cc.ko.vec loaded in 0:07:11.711635


  1%|          | 1/185 [00:00<00:25,  7.15it/s]

Finished run for neighbour size: 5 in 0:00:31.921732


...

Finished run for neighbour size: 50 in 0:00:50.226212


  0%|          | 0/272 [00:00<?, ?it/s]

Finished run for neighbour size: 75 in 0:00:53.489758


100%|██████████| 272/272 [00:56<00:00,  4.78it/s]


Finished run for neighbour size: 100 in 0:00:56.892230
Finished run for language: Turkish in 0:12:04.380668
Finished run for vector type: cc in 1:38:45.024622


In [48]:
for key in new_columns:
    print(str(key) + ':' + str(len(new_columns[key])))

5:4937
10:4937
25:4937
50:4937
75:4937
100:4937


In [50]:
pd_dataset.head()

,c,i,id,l1_c,l1_i,language,type,code,l2_sim_cc_5,l2_sim_cc_10,l2_sim_cc_25,l2_sim_cc_50,l2_sim_cc_75,l2_sim_cc_100
2827,plans,projects,person_2799,plans,projectes,Catalan,RN,ca,0.315914,0.289572,0.261106,0.246446,0.235596,0.228922
2733,waste,lose,person_2421,malgastar,perdre,Catalan,RV,ca,0.197119,0.175892,0.164163,0.149970,0.149026,0.142875
2732,ending,end,person_1465,final,final,Catalan,RN,ca,0.591592,0.500977,0.443042,0.402888,0.387598,0.379977
2731,shot,view,person_1465,disparat,mostra,Catalan,RN,ca,0.210400,0.187495,0.185070,0.189705,0.167300,0.161793
2730,do,join,person_1465,fer,unir,Catalan,RV,ca,0.225478,0.194971,0.201452,0.197570,0.192571,0.189414


In [52]:
for key in new_columns:
    pd_dataset['l1_sim_cc_' + str(key)] = new_columns[key]

In [54]:
pd_dataset.to_csv('../data/ms_final_experiments.csv', index = False)

In [55]:
back_up_col = {}
for key in new_columns.keys():
    if (key not in back_up_col):
        back_up_col[key] = []
    for row in new_columns[key]:
        back_up_col[key].append(str(row))

with open('final_similarity_dict.txt', 'w') as file:
     file.write(json.dumps(back_up_col)) # 